In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# remove the SettingWithCopyWarning

pd.options.mode.chained_assignment = None

In [ ]:
pg = "../data/07-clustered-graphs/alpha0.3_k10_res0.002.graphml"
G = nx.read_graphml(pg)
pdf = "../data/06-clustered-df/alpha0.3_k10_res0.002.pkl"
df = pd.read_pickle(pdf)

pcd = "../output/tables/cluster-explorer/SingleSolExplorer_alpha0.3_k10_res0.002.xlsx"
# first sheet is the cluster details
cluster_details = pd.read_excel(pcd, sheet_name=0)

# Sonification

- Piano: 0-7 (e.g., Acoustic Grand Piano is 0)
- Chromatic Percussion: 8-15 (e.g., Music Box is 10)
- Organ: 16-23 (e.g., Church Organ is 19)
- Guitar: 24-31 (e.g., Acoustic Guitar (nylon) is 24)
- Bass: 32-39 (e.g., Electric Bass (finger) is 33)
- Strings: 40-47 (e.g., Violin is 40, String Ensemble 1 is 48)
- Ensemble: 48-55 (e.g., String Ensemble 1 is 48)
- Brass: 56-63 (e.g., Trumpet is 56)
- Reed: 64-71 (e.g., Soprano Sax is 64)
- Pipe: 72-79 (e.g., Oboe is 68)
- Synth Lead: 80-87 (e.g., Lead 1 (square) is 80)
- Synth Pad: 88-95 (e.g., Pad 1 (new age) is 88)
- Synth Effects: 96-103
- Ethnic: 104-111 (e.g., Sitar is 104)
- Percussive: 112-119 (e.g., Steel Drums is 114)
- Sound Effects: 120-127 (e.g., Helicopter is 125)


In [ ]:
pdf = pivot_df.reset_index(drop=True)
pdf.columns = [name.split(";", 1)[0] if ";" in name else name for name in pdf.columns]
pdf.iloc[0:5, 0:5]

,0: mgkg,10: extrapyramidal,11: fluoxetine,12: social,13: fluoxetine
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0


In [ ]:
from midiutil import MIDIFile
import pandas as pd
import numpy as np


def create_sonified_midi(
    pdf,
    column_name="3: astrocytes",
    min_pitch=60,
    max_pitch=127,
    volume_range=(70, 127),
    filename="sonified_data.mid",
    instrument=0,
):
    """
    Create a MIDI file from a DataFrame's values, mapping data to pitch and volume.

    Parameters:
    - pdf: DataFrame containing the data to be sonified.
    - column_name: The column in the DataFrame to be used for sonification.
    - min_pitch: The minimum pitch value for the lowest data point (default 60).
    - max_pitch: The maximum pitch value for the highest data point (default 127).
    - volume_range: A tuple representing the minimum and maximum volume (default (70, 127)).
    - filename: The name of the output MIDI file (default "sonified_data.mid").
    """
    midi_file = MIDIFile(1)  # Create a new MIDI file with one track
    track = 0
    time = 0
    midi_file.addTrackName(track, time, "Track")
    midi_file.addTempo(track, time, 120)

    channel = 0
    instrument = instrument  # Set to your desired instrument
    midi_file.addProgramChange(track, channel, time, instrument)

    duration = 0.5  # Duration of each note in beats

    for index, row in pdf.iterrows():
        # Map the data value to the pitch range
        data_value = row[column_name]
        if data_value > 0:
            pitch = max(
                int(np.interp(data_value, [0, 1], [min_pitch, max_pitch])), min_pitch
            )
        else:
            continue  # Skip rows with 0 or negative values

        # Map the data value to the volume range
        volume = int(np.interp(data_value, [0, 1], volume_range))

        midi_file.addNote(track, channel, pitch, time, duration, volume)
        time += duration
        # print(volume)
    # Write to disk
    with open(filename, "wb") as output_file:
        midi_file.writeFile(output_file)
    print(f"MIDI file '{filename}' has been created.")

In [ ]:
# Assuming you have a DataFrame named 'pdf' ready
# pdf = pd.DataFrame({'0: memory': np.random.rand(10)})  # Example, replace with your DataFrame

create_sonified_midi(
    pdf,
    column_name="3: astrocytes",
    min_pitch=30,
    max_pitch=127,
    volume_range=(50, 127),
    filename="enhanced_sonified_data.mid",
    instrument=0,
)

KeyError: '3: astrocytes'

In [ ]:
import os
from midi2audio import FluidSynth
from IPython.display import Audio

# Convert MIDI to WAV using FluidSynth
soundfont = "/Users/jlq293/Projects/sound/GeneralUser GS 1.471/GeneralUser GS v1.471.sf2"  # Update with the path to your soundfont file
midi_file = (
    "/Users/jlq293/Projects/Study-1-Bibliometrics/notebooks/enhanced_sonified_data.mid"
)
wav_file = "sonified_data.wav"

# Initialize FluidSynth with the soundfont
fs = FluidSynth(soundfont)

# Convert MIDI to WAV
fs.midi_to_audio(midi_file, wav_file)

# Play the WAV file
Audio(wav_file)

FluidSynth runtime version 2.3.4
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'sonified_data.wav'..


In [ ]:
f